# Assignment #4 : Decision Tree & Ensemble Algorithms

## STD IDs: 9931155 & 40133693

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from DesicionTree import DT, preprocess

DS_path="Dataset/Covid19HDDT.csv"

max_depth=[2, 3, 4, 5]
cut_off=[10, 50, 500]
n_iteration=10

data, label = preprocess.read_dataset(DS_path)

# Remove most correlatied feature from data
data = preprocess.remove_correlated_with_label(
    data=data, label=label, threshold=0.4
)

Create funtion to get data, label, max_depth, cut_off size
and n_itereration and return accuracy of model with input parameters


In [5]:
def run_HDDT(
    data: np.ndarray,
    label: np.ndarray,
    n_iteration: int,
    max_depth: int = None,
    cut_off: int = None,
    print_tabel: bool = False,
):
    acc = np.empty((0, 5))
    for _ in range(n_iteration):
        # Split data to train and test with sklearn
        data_train, data_test, label_train, label_test = train_test_split(
            data, label, test_size=0.3, stratify=label
        )

        # Create model
        model = DT(max_depth=max_depth, cut_off_size=cut_off)
        model.fit(data_train, label_train)
        predicted = model.predict(data_test)
        acc = np.vstack((acc, preprocess.accuracy(predicted, label_test)))

    label_table = ["Precision", "Recall", "F-measure", "AUC", "G-mean"]
    if print_tabel:
        print(
            tabulate(
                acc,
                headers=label_table,
                tablefmt="fancy_grid",
                showindex="always",
                floatfmt=".4f",
            )
        )
    for metrinc in label_table:
        print(
            f"avg {metrinc}:",
            f"{np.mean(acc[:, label_table.index(metrinc)]).round(4)}",  
        )


Run the HDDT algorithm for each max_depth and cut_off size and print accuracy table for each of them

In [6]:
for depth in max_depth:
    for cut in cut_off:
        print(f"max_depth: {depth}, cut_off: {cut}\n")
        run_HDDT(
            data,
            label,
            n_iteration=n_iteration,
            max_depth=depth,
            cut_off=cut,
        )
        print(f"{'='*50}")

print(f"max_depth: None, cut_off: None\n")
run_HDDT(
    data,
    label,
    n_iteration=n_iteration,
)


max_depth: 2, cut_off: 10

avg Precision: 0.9075
avg Recall: 0.9075
avg F-measure: 0.9075
avg AUC: 0.8995
avg G-mean: 0.9075
max_depth: 2, cut_off: 50

avg Precision: 0.9083
avg Recall: 0.9083
avg F-measure: 0.9083
avg AUC: 0.8963
avg G-mean: 0.9083
max_depth: 2, cut_off: 500

avg Precision: 0.9084
avg Recall: 0.9084
avg F-measure: 0.9084
avg AUC: 0.8944
avg G-mean: 0.9084
max_depth: 3, cut_off: 10

avg Precision: 0.9065
avg Recall: 0.9065
avg F-measure: 0.9065
avg AUC: 0.8881
avg G-mean: 0.9065
max_depth: 3, cut_off: 50

avg Precision: 0.9063
avg Recall: 0.9063
avg F-measure: 0.9063
avg AUC: 0.8901
avg G-mean: 0.9063
max_depth: 3, cut_off: 500

avg Precision: 0.9064
avg Recall: 0.9064
avg F-measure: 0.9064
avg AUC: 0.8803
avg G-mean: 0.9064
max_depth: 4, cut_off: 10

avg Precision: 0.9038
avg Recall: 0.9038
avg F-measure: 0.9038
avg AUC: 0.8854
avg G-mean: 0.9038
max_depth: 4, cut_off: 50

avg Precision: 0.9028
avg Recall: 0.9028
avg F-measure: 0.9028
avg AUC: 0.8794
avg G-mean: 0.902